In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import torch

In [ ]:
from data_utils import get_data_for_global_evaluation
from global_model import GlobalModel
from evaluate_script import evaluate_on_test_dataset

# Global model evaluation

In [ ]:
model = GlobalModel(device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

# Évaluer sur l'ensemble de test
#test_dir = "dataset/new_test"
test_dir = "dataset/test"

results = evaluate_on_test_dataset(model, test_dir)